In [3]:
import sys
sys.path.append('..')

from ast2vec.python_ast_utils import *
import ast
from tqdm import tqdm
from ast2vec import tree as tree
from ast2vec.ast2vec import encode_trees, load_model
from ast2vec import tree_grammar as tree_grammar
import pandas as pd

from tqdm import tqdm
tqdm.pandas()
from torch.nn.utils.rnn import pad_sequence
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
def convert_to_tree(src):
    parser = tree_grammar.TreeParser(grammar)
    try:
        ast_tree = ast.parse(src, mode='exec')
        x = ast_to_tree(ast_tree)
        parser.parse_tree(x)
        return x
    except:
        return None

In [6]:
# data = pd.read_pickle('data/python_submits_test.pkl')

In [4]:
data['tree'] = data['code'].progress_apply(convert_to_tree)

100%|███████████████████████████████████████████████████████████████████████████████| 488/488 [00:00<00:00, 1963.12it/s]


In [5]:
model = load_model('ast2vec/ast2vec.pt')

In [7]:
import multiprocessing
import time
multiprocessing.cpu_count()

8

In [16]:
start_time = time.time()
X_multi = encode_trees(model, data['tree'].values, verbose=True, n_jobs=8)
finish_time = time.time()

Worker: 0: 100%|████████████████████████████████████████████████████████████████████████| 61/61 [00:01<00:00, 35.41it/s]
Worker: 1: 61it [00:01, 33.34it/s]                                                                                      
Worker: 2: 61it [00:01, 33.17it/s]                                                                                      
Worker: 3: 61it [00:01, 31.69it/s]                                                                                      
Worker: 6: 61it [00:01, 32.85it/s]                                                                                      
Worker: 5: 61it [00:01, 32.10it/s]                                                                                      
Worker: 4: 61it [00:01, 31.30it/s]                                                                                      
Worker: 7: 61it [00:01, 31.65it/s]                                                                                      


In [17]:
print(finish_time-start_time)

2.152925968170166


In [12]:
X_one = encode_trees(model, data['tree'].values, verbose=True, n_jobs=1)

100%|████████████████████████████████████████████████████████████████████████████████| 488/488 [00:04<00:00, 121.97it/s]


## Sanity check

In [47]:
data = pd.read_pickle('data/python_submits.pkl')

In [48]:
data = data.reset_index(drop=True)

In [49]:
data_test = data[data.user_id == 39136]

In [50]:
data_test['tree'] = data_test['code'].progress_apply(convert_to_tree)

100%|███████████████████████████████████████████████████████████████████████████████| 244/244 [00:00<00:00, 1965.07it/s]
/tmp/ipykernel_8945/2037145316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['tree'] = data_test['code'].progress_apply(convert_to_tree)


In [13]:
model = load_model('ast2vec/ast2vec.pt')

In [52]:
test_array = encode_trees(model, data_test['tree'].values, verbose=True, n_jobs=8) 

Worker: 0: 31it [00:01, 28.54it/s]                                                                                      
Worker: 1: 31it [00:01, 27.64it/s]                                                                                      
Worker: 2: 31it [00:01, 28.25it/s]                                                                                      
Worker: 3: 31it [00:01, 25.60it/s]                                                                                      
Worker: 4: 100%|████████████████████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 25.77it/s]
Worker: 5: 30it [00:01, 24.62it/s]                                                                                      
Worker: 6: 30it [00:01, 25.58it/s]                                                                                      
Worker: 7: 30it [00:01, 26.29it/s]                                                                                      


In [53]:
test_array = torch.from_numpy(test_array)

In [54]:
x_emb = torch.load('data/submits_emb.pt')

In [56]:
torch.allclose(x_emb[data_test.submit_index.values], test_array)

True

In [7]:
from data_loading import CustomDataset,CollateClass, make_dataloader
import wandb
import yaml

In [8]:
problems = pd.read_pickle('data/python_statements.pkl')
submits_embeddings = torch.load('data/submits_emb.pt')
data = pd.read_pickle('data/python_submits.pkl')

In [9]:
config = yaml.safe_load(open("config.yaml", "r"))
wandb.init(
    project="knowledge_tracing",
    config=config,
    mode='disabled'
)

collate_fn = CollateClass(device='cpu', max_len=20)
dataloader = make_dataloader([52422], data.set_index('user_id'), submits_embeddings, problems, shuffle=False, collate_fn=collate_fn)

In [17]:
code,prompt, score = next(iter(dataloader))

In [10]:
test_data = data[data.user_id == 52422].sort_values(by='timestamp')

In [11]:
test_data['tree'] = test_data['code'].progress_apply(convert_to_tree)

100%|███████████████████████████████████████████████████████████████████████████████| 274/274 [00:00<00:00, 2271.18it/s]


In [14]:
test_array = encode_trees(model, test_data['tree'].values, verbose=True, n_jobs=8) 

Worker: 1: 35it [00:01, 32.90it/s]                                                                                      
Worker: 0: 35it [00:01, 27.75it/s]                                                                                      
Worker: 2: 100%|████████████████████████████████████████████████████████████████████████| 34/34 [00:01<00:00, 27.54it/s]
Worker: 7: 34it [00:01, 29.40it/s]                                                                                      
Worker: 6: 34it [00:01, 26.67it/s]                                                                                      
Worker: 5: 34it [00:01, 25.49it/s]                                                                                      
Worker: 3: 34it [00:01, 23.45it/s]                                                                                      
Worker: 4: 34it [00:01, 18.67it/s]                                                                                      


In [23]:
import numpy as np

np.allclose(test_array[:20],code)

True